In [3]:
import pandas as pd
import numpy as np
import sys
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

In [4]:
sys.path.append(os.path.abspath('../scripts'))

import preprocess
import feature_engineering
import model_training

ModuleNotFoundError: No module named 'feature_engineering'

In [ ]:
credit_df = preprocess.load_data('../data/creditcard.csv')
credit_df = preprocess.clean_credit_card_data(credit_df)

In [ ]:
X_credit = credit_df.drop('Class', axis=1)
y_credit = credit_df['Class']

In [ ]:
X_credit_train, X_credit_test, y_credit_train, y_credit_test = model_training.split_data(X_credit, y_credit)


In [ ]:
scaler = StandardScaler()
X_credit_train[['Time', 'Amount']] = scaler.fit_transform(X_credit_train[['Time', 'Amount']])
X_credit_test[['Time', 'Amount']] = scaler.transform(X_credit_test[['Time', 'Amount']])

In [ ]:
print("Class distribution before SMOTE:", y_credit_train.value_counts())
smote = SMOTE(random_state=42)
X_credit_train_res, y_credit_train_res = smote.fit_resample(X_credit_train, y_credit_train)
print("Class distribution after SMOTE:", y_credit_train_res.value_counts())

In [ ]:
lr_credit_model = model_training.train_logistic_regression(X_credit_train_res, y_credit_train_res)
lr_credit_metrics = model_training.evaluate_model(lr_credit_model, X_credit_test, y_credit_test, "Logistic Regression (Credit Card)")

In [ ]:
rf_credit_model = model_training.train_random_forest(X_credit_train_res, y_credit_train_res)
rf_credit_metrics = model_training.evaluate_model(rf_credit_model, X_credit_test, y_credit_test, "Random Forest (Credit Card)")

In [ ]:
fraud_df = preprocess.load_data('../data/Fraud_Data.csv')
fraud_df = preprocess.clean_fraud_data(fraud_df)

In [ ]:
fraud_df = feature_engineering.create_time_features(fraud_df)
fraud_df = feature_engineering.time_since_signup(fraud_df)
fraud_df = fraud_df.drop(['user_id', 'signup_time', 'purchase_time', 'device_id', 'ip_address'], axis=1)

In [ ]:
X_fraud = fraud_df.drop('class', axis=1)
y_fraud = fraud_df['class']

In [ ]:
categorical_features = ['source', 'browser', 'sex']
numerical_features = ['purchase_value', 'age', 'hour_of_day', 'day_of_week', 'time_since_signup']

In [ ]:
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = model_training.split_data(X_fraud, y_fraud)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])


In [ ]:
lr_fraud_pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])
lr_fraud_pipeline.fit(X_fraud_train, y_fraud_train)
lr_fraud_metrics = model_training.evaluate_model(lr_fraud_pipeline, X_fraud_test, y_fraud_test, "Logistic Regression (E-commerce)")

In [ ]:
rf_fraud_pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42, n_estimators=100, class_weight='balanced'))
])
rf_fraud_pipeline.fit(X_fraud_train, y_fraud_train)
rf_fraud_metrics = model_training.evaluate_model(rf_fraud_pipeline, X_fraud_test, y_fraud_test, "Random Forest (E-commerce)")

In [ ]:
print("\n--- Model Comparison ---")
print("\nCredit Card Dataset:")
print(f"Logistic Regression -> F1: {lr_credit_metrics['f1_score']:.4f}, AUC-PR: {lr_credit_metrics['auc_pr']:.4f}")
print(f"Random Forest -> F1: {rf_credit_metrics['f1_score']:.4f}, AUC-PR: {rf_credit_metrics['auc_pr']:.4f}")

In [ ]:
print("\nE-commerce Dataset:")
print(f"Logistic Regression -> F1: {lr_fraud_metrics['f1_score']:.4f}, AUC-PR: {lr_fraud_metrics['auc_pr']:.4f}")
print(f"Random Forest -> F1: {rf_fraud_metrics['f1_score']:.4f}, AUC-PR: {rf_fraud_metrics['auc_pr']:.4f}")
